# Multivariate Forecasting with Chronos-2 (Google Colab)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/Chronos2_Colab_Experiments.ipynb)

## Research Questions

1. Do multivariate (MV) methods produce better predictions than univariate (UV) ones when foundation models are used?
2. Is MV forecasting accuracy better for stocks versus interest rates?
3. Is MV forecasting better when both stocks and interest rates are forecast together?
4. Can we build a large-scale "world" forecasting model?

---

**Complete implementation per README specifications - Optimized for Google Colab**

## 🔧 Setup: Install Dependencies

**Run this first!** Installs all required packages.

In [ ]:
# Install required packages
!pip install -q chronos-forecasting torch pandas numpy yfinance matplotlib seaborn tqdm python-dateutil

print("✓ All packages installed!")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import json
import os
import yfinance as yf
from chronos import Chronos2Pipeline

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 📊 Data Loader Class

Downloads Magnificent-7 stocks and FRED interest rates.

In [ ]:
class DataLoader:
    """Load and prepare financial time series data for Chronos-2 experiments."""
    
    def __init__(self):
        # Magnificent-7 stocks
        self.mag7_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA"]
        
        # FRED interest rate series (10 maturities per README)
        self.fred_series = {
            "DGS3MO": "3-Month",
            "DGS6MO": "6-Month",
            "DGS1": "1-Year",
            "DGS2": "2-Year",
            "DGS3": "3-Year",
            "DGS5": "5-Year",
            "DGS7": "7-Year",
            "DGS10": "10-Year",
            "DGS20": "20-Year",
            "DGS30": "30-Year"
        }
    
    def download_stocks(self, start_date="2000-01-01", end_date=None):
        """Download Magnificent-7 stock data from Yahoo Finance."""
        if end_date is None:
            end_date = datetime.now().strftime("%Y-%m-%d")
        
        print(f"Downloading Magnificent-7 stocks ({start_date} to {end_date})...")
        
        df = yf.download(self.mag7_tickers, start=start_date, end=end_date, 
                       auto_adjust=True, progress=False)
        
        # Extract close prices
        if isinstance(df.columns, pd.MultiIndex):
            df_close = df["Close"].copy()
        else:
            df_close = df.copy()
        
        # Reset index and clean
        df_close = df_close.reset_index()
        df_close = df_close.rename(columns={"Date": "timestamp"})
        df_close["timestamp"] = pd.to_datetime(df_close["timestamp"])
        
        # Business day frequency with forward fill
        df_close = df_close.set_index("timestamp").asfreq("B").ffill().reset_index()
        
        print(f"✓ Downloaded {len(df_close)} rows for {len(self.mag7_tickers)} stocks")
        return df_close
    
    def download_interest_rates(self, start_date="2000-01-01", end_date=None):
        """Download FRED interest rate data."""
        if end_date is None:
            end_date = datetime.now().strftime("%Y-%m-%d")
        
        print(f"\nDownloading FRED interest rates ({start_date} to {end_date})...")
        
        all_rates = []
        
        for ticker, name in self.fred_series.items():
            try:
                # Try direct FRED CSV download
                url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={ticker}"
                df = pd.read_csv(url, parse_dates=["DATE"])
                df = df.rename(columns={"DATE": "timestamp", ticker: ticker})
                df = df.set_index("timestamp")
                df = df[(df.index >= start_date) & (df.index <= end_date)]
                
                # Replace '.' with NaN
                df[ticker] = pd.to_numeric(df[ticker], errors='coerce')
                
                all_rates.append(df[ticker])
                print(f"✓ Downloaded {name} ({ticker})")
                
            except Exception as e:
                print(f"✗ Failed to download {ticker}: {e}")
        
        if not all_rates:
            raise ValueError("Could not download any interest rate data")
        
        # Combine all rates
        df_rates = pd.concat(all_rates, axis=1)
        df_rates = df_rates.reset_index()
        df_rates = df_rates.rename(columns={"index": "timestamp"})
        df_rates["timestamp"] = pd.to_datetime(df_rates["timestamp"])
        
        # Business day frequency with forward fill
        df_rates = df_rates.set_index("timestamp").asfreq("B").ffill().reset_index()
        
        print(f"✓ Combined {len(df_rates)} rows for {len(all_rates)} interest rates")
        return df_rates
    
    def download_combined(self, start_date="2000-01-01", end_date=None):
        """Download and combine stocks + interest rates (K=17)."""
        print("\n" + "="*60)
        print("DOWNLOADING COMBINED DATASET (Stocks + Interest Rates)")
        print("="*60)
        
        stocks = self.download_stocks(start_date, end_date)
        rates = self.download_interest_rates(start_date, end_date)
        
        # Merge on timestamp
        combined = pd.merge(stocks, rates, on="timestamp", how="inner")
        
        print(f"\n✓ Combined dataset: {len(combined)} rows, {len(combined.columns)-1} series (K=17)")
        return combined

print("✓ DataLoader class defined")

## 📐 Metrics Calculator

Implements RMSE and MAPE per README formulas.

In [ ]:
class MetricsCalculator:
    """Calculate forecast error metrics per README specifications."""
    
    @staticmethod
    def calculate_rmse(actual, predicted):
        """
        RMSE = [1/m * sum((x - y)^2)]^(1/2)
        """
        m = len(actual)
        squared_errors = (actual - predicted) ** 2
        rmse = np.sqrt(np.mean(squared_errors))
        return float(rmse)
    
    @staticmethod
    def calculate_mape(actual, predicted):
        """
        MAPE = 1/m * sum(|x - y| / |x|) * 100
        """
        # Avoid division by zero
        mask = actual != 0
        if not mask.any():
            return np.inf
        
        actual_masked = actual[mask]
        predicted_masked = predicted[mask]
        
        absolute_percentage_errors = np.abs((actual_masked - predicted_masked) / actual_masked)
        mape = np.mean(absolute_percentage_errors) * 100
        return float(mape)
    
    @staticmethod
    def calculate_all_metrics(actual, predicted):
        return {
            'rmse': MetricsCalculator.calculate_rmse(actual, predicted),
            'mape': MetricsCalculator.calculate_mape(actual, predicted)
        }
    
    @staticmethod
    def compare_uv_mv_metrics(uv_metrics, mv_metrics):
        rmse_improvement = ((uv_metrics['rmse'] - mv_metrics['rmse']) / uv_metrics['rmse']) * 100
        mape_improvement = ((uv_metrics['mape'] - mv_metrics['mape']) / uv_metrics['mape']) * 100
        
        return {
            'uv_rmse': uv_metrics['rmse'],
            'mv_rmse': mv_metrics['rmse'],
            'rmse_improvement_pct': rmse_improvement,
            'uv_mape': uv_metrics['mape'],
            'mv_mape': mv_metrics['mape'],
            'mape_improvement_pct': mape_improvement,
            'mv_better_rmse': mv_metrics['rmse'] < uv_metrics['rmse'],
            'mv_better_mape': mv_metrics['mape'] < uv_metrics['mape']
        }

print("✓ MetricsCalculator class defined")

## 🚀 Chronos-2 Experiment Runner

Main class for running UV vs MV experiments.

In [ ]:
class ChronosExperimentRunner:
    """Run Chronos-2 forecasting experiments per README specifications."""
    
    def __init__(self, model_size="base", device="cuda"):
        self.model_size = model_size
        self.metrics_calc = MetricsCalculator()
        
        # Load Chronos-2 model
        print(f"Loading Chronos-2 {model_size} model...")
        device = device if torch.cuda.is_available() else "cpu"
        self.pipeline = Chronos2Pipeline.from_pretrained(
            f"amazon/chronos-2-{model_size}",
            device_map=device
        )
        print(f"✓ Model loaded on {device}")
    
    def prepare_data_for_date(self, df, target_date, n, m):
        """Prepare context and test data for a specific date."""
        df = df.copy()
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.sort_values('timestamp')
        
        # Find index of target date
        target_idx = df[df['timestamp'] <= target_date].index[-1]
        
        # Context: n days before target_date
        context_start_idx = max(0, target_idx - n + 1)
        context_df = df.iloc[context_start_idx:target_idx+1].copy()
        
        # Test: m days after target_date
        test_start_idx = target_idx + 1
        test_end_idx = min(len(df), test_start_idx + m)
        test_df = df.iloc[test_start_idx:test_end_idx].copy()
        
        return context_df, test_df
    
    def forecast_univariate(self, context_df, target_column, prediction_length):
        """Univariate forecast for single series."""
        context_uv = context_df[["item_id", "timestamp", target_column]].copy()
        
        future_df = pd.DataFrame({
            "item_id": context_uv["item_id"].iloc[-1],
            "timestamp": pd.date_range(
                start=context_uv["timestamp"].iloc[-1] + pd.Timedelta(days=1),
                periods=prediction_length,
                freq="B"
            )
        })
        
        pred_df = self.pipeline.predict_df(
            context_uv,
            future_df=future_df,
            prediction_length=prediction_length,
            quantile_levels=[0.1, 0.5, 0.9],
            id_column="item_id",
            timestamp_column="timestamp",
            target=target_column
        )
        
        return pred_df
    
    def forecast_multivariate(self, context_df, target_column, prediction_length):
        """
        Multivariate forecast using all series.
        IMPORTANT: No future covariate values - Chronos-2 learns from historical relationships.
        """
        # Create future_df with ONLY timestamp and item_id
        future_df = pd.DataFrame({
            "item_id": context_df["item_id"].iloc[-1],
            "timestamp": pd.date_range(
                start=context_df["timestamp"].iloc[-1] + pd.Timedelta(days=1),
                periods=prediction_length,
                freq="B"
            )
        })
        
        pred_df = self.pipeline.predict_df(
            context_df,
            future_df=future_df,
            prediction_length=prediction_length,
            quantile_levels=[0.1, 0.5, 0.9],
            id_column="item_id",
            timestamp_column="timestamp",
            target=target_column
        )
        
        return pred_df
    
    def run_single_experiment(self, df, target_date, n, m, series_name, dataset_type):
        """Run single UV vs MV experiment."""
        # Prepare data
        context_df, test_df = self.prepare_data_for_date(df, target_date, n, m)
        
        if len(test_df) < m:
            return None
        
        # Add item_id if not present
        if 'item_id' not in context_df.columns:
            context_df['item_id'] = dataset_type
            test_df['item_id'] = dataset_type
        
        # Run forecasts
        uv_pred = self.forecast_univariate(context_df, series_name, m)
        mv_pred = self.forecast_multivariate(context_df, series_name, m)
        
        # Get actual values
        actual = test_df.set_index("timestamp")[series_name].values[:m]
        uv_values = uv_pred[series_name].values[:m]
        mv_values = mv_pred[series_name].values[:m]
        
        # Calculate metrics
        uv_metrics = self.metrics_calc.calculate_all_metrics(actual, uv_values)
        mv_metrics = self.metrics_calc.calculate_all_metrics(actual, mv_values)
        comparison = self.metrics_calc.compare_uv_mv_metrics(uv_metrics, mv_metrics)
        
        result = {
            'dataset': dataset_type,
            'series': series_name,
            'target_date': target_date.strftime('%Y-%m-%d'),
            'n': n,
            'm': m,
            'alpha': n / 252,
            **comparison,
            'actual_values': actual.tolist(),
            'uv_predictions': uv_values.tolist(),
            'mv_predictions': mv_values.tolist()
        }
        
        return result

print("✓ ChronosExperimentRunner class defined")

## 📥 Step 1: Download Data

Download Magnificent-7 stocks and FRED interest rates.

In [ ]:
# Initialize data loader
loader = DataLoader()

# Download all datasets
print("Downloading data from 2000-01-01 to present...\n")

stocks_df = loader.download_stocks(start_date="2000-01-01")
rates_df = loader.download_interest_rates(start_date="2000-01-01")
combined_df = loader.download_combined(start_date="2000-01-01")

print("\n" + "="*60)
print("DATA SUMMARY")
print("="*60)
print(f"Stocks: {stocks_df.shape[0]} rows, {stocks_df.shape[1]-1} series (K=7)")
print(f"Interest Rates: {rates_df.shape[0]} rows, {rates_df.shape[1]-1} series (K=10)")
print(f"Combined: {combined_df.shape[0]} rows, {combined_df.shape[1]-1} series (K=17)")

In [ ]:
# Preview data
print("\nStocks (Magnificent-7):")
display(stocks_df.head())

print("\nInterest Rates (FRED):")
display(rates_df.head())

## 🧪 Step 2: Single Test Experiment

Per README: n=252, m=21, t=03/31/2025, test with NVDA

In [ ]:
# Initialize experiment runner
runner = ChronosExperimentRunner(model_size="base")

# Prepare test data
stocks_df['item_id'] = 'stocks'
test_date = datetime(2025, 3, 31)
n = 252  # 1 year
m = 21   # 1 month

print(f"\nTest Experiment:")
print(f"  Date: {test_date.strftime('%Y-%m-%d')}")
print(f"  History (n): {n} days")
print(f"  Forecast (m): {m} days")
print(f"  Series: NVDA")

In [ ]:
# Run test experiment
result = runner.run_single_experiment(
    df=stocks_df,
    target_date=test_date,
    n=n,
    m=m,
    series_name='NVDA',
    dataset_type='stocks'
)

if result:
    print("\n" + "="*60)
    print("TEST RESULTS")
    print("="*60)
    print(f"\nUnivariate (UV):")
    print(f"  RMSE: {result['uv_rmse']:.4f}")
    print(f"  MAPE: {result['uv_mape']:.2f}%")
    print(f"\nMultivariate (MV):")
    print(f"  RMSE: {result['mv_rmse']:.4f}")
    print(f"  MAPE: {result['mv_mape']:.2f}%")
    print(f"\nImprovement:")
    print(f"  RMSE: {result['rmse_improvement_pct']:.2f}%")
    print(f"  MAPE: {result['mape_improvement_pct']:.2f}%")
    print(f"  MV Better: {result['mv_better_mape']}")
else:
    print("Test failed - check data availability")

In [ ]:
# Visualize test results
if result:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    timestamps = pd.date_range(start=test_date, periods=m+1, freq='B')[1:]
    
    ax.plot(timestamps, result['actual_values'], 'ko-', label='Actual', linewidth=2, markersize=6)
    ax.plot(timestamps, result['uv_predictions'], 'b^--', label='UV Forecast', linewidth=2, markersize=6)
    ax.plot(timestamps, result['mv_predictions'], 'rs--', label='MV Forecast', linewidth=2, markersize=6)
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('NVDA Stock Price ($)', fontsize=12)
    ax.set_title(f'NVDA Forecast: UV vs MV (n={n}, m={m})', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print("✓ Test experiment completed!")

## 📊 Step 3: Run Multiple Experiments

**Note:** Full experiments take a long time. Start with a small subset for testing.

For full experiments per README:
- α = {0.5, 1, 2, 3}
- m = {21, 63}
- Monthly rolling from 2000-2025
- All 7 stocks or 10 rates

In [ ]:
# Example: Run experiments for 2 stocks, 1 parameter combo, quarterly 2024
test_stocks = ['AAPL', 'NVDA']
alpha = 1.0  # n = 252
m = 21       # 1 month

# Generate quarterly dates in 2024
forecast_dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='QS')

results = []

print(f"\nRunning experiments:")
print(f"  Series: {test_stocks}")
print(f"  n={int(alpha*252)}, m={m}")
print(f"  Dates: {len(forecast_dates)} quarterly dates in 2024")
print(f"  Total: {len(test_stocks) * len(forecast_dates)} experiments\n")

for series_name in test_stocks:
    for target_date in tqdm(forecast_dates, desc=f"Forecasting {series_name}"):
        try:
            result = runner.run_single_experiment(
                df=stocks_df,
                target_date=target_date,
                n=int(alpha*252),
                m=m,
                series_name=series_name,
                dataset_type='stocks'
            )
            if result:
                results.append(result)
        except Exception as e:
            print(f"Error: {series_name}, {target_date}: {e}")

print(f"\n✓ Completed {len(results)} experiments")

In [ ]:
# Analyze results
if results:
    df_results = pd.DataFrame(results)
    
    print("\n" + "="*60)
    print("EXPERIMENT SUMMARY")
    print("="*60)
    
    mv_win_rate = (df_results['mv_better_mape'].sum() / len(df_results)) * 100
    print(f"\nMV Win Rate (MAPE): {mv_win_rate:.1f}%")
    
    avg_mape_improvement = df_results['mape_improvement_pct'].mean()
    print(f"Average MAPE Improvement: {avg_mape_improvement:.2f}%")
    
    print("\nBy Series:")
    series_summary = df_results.groupby('series').agg({
        'mv_better_mape': 'mean',
        'mape_improvement_pct': 'mean',
        'uv_mape': 'mean',
        'mv_mape': 'mean'
    }).round(2)
    series_summary.columns = ['MV Win Rate', 'Avg MAPE Improvement %', 'Avg UV MAPE', 'Avg MV MAPE']
    display(series_summary)

In [ ]:
# Visualize results
if results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # MV Win Rate by Series
    win_rates = df_results.groupby('series')['mv_better_mape'].mean() * 100
    axes[0].bar(win_rates.index, win_rates.values, color='steelblue')
    axes[0].set_ylabel('MV Win Rate (%)', fontsize=12)
    axes[0].set_title('MV Win Rate by Series', fontsize=13, fontweight='bold')
    axes[0].axhline(y=50, color='r', linestyle='--', alpha=0.5)
    axes[0].grid(True, alpha=0.3)
    
    # MAPE Improvement
    improvements = df_results.groupby('series')['mape_improvement_pct'].mean()
    axes[1].bar(improvements.index, improvements.values, color='coral')
    axes[1].set_ylabel('Avg MAPE Improvement (%)', fontsize=12)
    axes[1].set_title('Average MAPE Improvement', fontsize=13, fontweight='bold')
    axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 💾 Save Results

Save to Google Drive (optional)

In [ ]:
# Save results to JSON
if results:
    # Save to Colab's local storage
    with open('experiment_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    print("✓ Saved to experiment_results.json")
    
    # Save summary to CSV
    summary_results = []
    for r in results:
        summary = {k: v for k, v in r.items() 
                  if k not in ['actual_values', 'uv_predictions', 'mv_predictions']}
        summary_results.append(summary)
    
    df_summary = pd.DataFrame(summary_results)
    df_summary.to_csv('experiment_summary.csv', index=False)
    print("✓ Saved to experiment_summary.csv")
    
    # Download files
    from google.colab import files
    files.download('experiment_results.json')
    files.download('experiment_summary.csv')

## 📝 Research Question Answers

Based on your experimental results, answer:

1. **Do MV methods produce better predictions than UV with foundation models?**
   - [Fill in based on MV win rate and MAPE improvements]

2. **Is MV forecasting accuracy better for stocks versus interest rates?**
   - [Compare stocks vs rates experiments]

3. **Is MV forecasting better when both stocks and interest rates are forecast together?**
   - [Compare combined (K=17) vs separate experiments]

4. **Can we build a large-scale "world" forecasting model?**
   - [Discuss scalability based on K=17 results]

## 🎯 Next Steps

**For full experiments per README:**

1. Modify the experiment loop to use:
   - All 7 stocks: `['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA']`
   - All α values: `[0.5, 1.0, 2.0, 3.0]`
   - Both m values: `[21, 63]`
   - Monthly rolling from 2000-2025

2. Run experiments for:
   - Stocks (K=7)
   - Interest rates (K=10)
   - Combined (K=17)

3. Analyze and compare results

**Warning:** Full experiments will take many hours. Use GPU runtime for faster execution.